In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

import matplotlib.pyplot as plt
import numpy as np
import seaborn

import ising_model
import exact_ising_model as exact
import plotting
import analysis
import simulation

# Metropolis
Lattice Size 4, Temperature 8.0
Energy per Site: -0.2610626220703125 +/- 0.0018771324458461633
Magnetization per Site: -1.9073486328125e-06 +/- 0.002077072882542814
m (<|M|>/N): 0.26807212829589844 +/- 0.0010193575963454566
Energy Squared: 54.25830078125 +/- 0.44047312300298547
Magnetization Squared: 28.64300537109375 +/- 0.18814095808833015
Heat Capacity per Site: 0.03599535199100501 +/- 0.00038559787097370484

# Wolff
Lattice Size 4, Temperature 8.0
Energy per Site: -0.267791748046875 +/- 0.0032271910499994372
Magnetization per Site: -0.0035552978515625 +/- 0.00286548199467902
m (<|M|>/N): 0.2700157165527344 +/- 0.0012436922244417024
Energy Squared: 55.67626953125 +/- 0.6168333494425499
Magnetization Squared: 28.962158203125 +/- 0.23760101756071275
Heat Capacity per Site: 0.036492407917194214 +/- 0.0004084473137390334

# Corboz
Energy per site : -0.263511657715  +/-  0.00150418036608
Magnetization per site : 0.000926971435547  +/-  0.00199863921378
m : 0.268798828125  +/-  0.000809964714865
Energy squared : 54.658203125  +/-  0.391778044629
Magnetization squared : 28.7371826172  +/-  0.149540865285

In [ ]:
ising = ising_model.IsingModel(4, 1, 2.2, "hi", 70536)
ising.metropolis(False)
equilibrium = 5000
equi_energy = ising.energy_history[5000:] / ising.no_of_sites
equi_mag = ising.magnetization_history[5000:] / ising.no_of_sites

In [ ]:
bin_en_sq = analysis.binning(equi_energy**2, 8, "Energy Squared", show_plot=True)[-1]
print(bin_en_sq)
bin_en = analysis.binning(equi_energy, 8, "Energy", show_plot=True)[-1]
print(bin_en)
16/2.2*(np.mean(bin_en_sq) - np.mean(bin_en)**2)
# print(analysis.jackknife([bin_en, bin_en_sq], 8, analysis.heat_capacity, temperature=4, no_of_sites=16))

In [ ]:
simulation.uncoupled_simulation("ising", "wolff", 4, 1, "hi", 5000, 65536, 5)

In [ ]:
ising = ising_model.IsingModel(400, 1, 3, "hi", 9096)

In [ ]:
%lprun -f ising.wolff ising.wolff()

In [ ]:
sizes = []
for n in np.arange(1, 5, 0.2):
    ising = ising_model.IsingModel(4, 1, n, "hi", 9096)
    sizes.append(np.mean(ising.wolff()))

plt.plot(np.arange(1, 5, 0.2), sizes)

In [ ]:
sizes = [s / 16 for s in sizes]
plt.plot(np.arange(1, 5, 0.2), sizes)

In [ ]:
ising = ising_model.IsingModel(4, 1, 3, "hi", 9096)
sizes = ising.wolff()

In [ ]:
equi_e = ising.energy_history[5000:] / ising.no_of_sites
equi_m = ising.magnetization_history[5000:] / ising.no_of_sites

In [ ]:
analysis.binning(equi_e, "Energy", True)

In [ ]:
simulation.uncoupled_simulation("ising", "metropolis", 4, 1, 8, "hi", 5000, 65536, True)

In [ ]:
simulation.simulation_range("ising", "wolff", [4, 10, 20], 1, "lo", 5000, 4096, 2, 3)

In [ ]:
ising = ising_model.IsingModel(100, 1, 1, "lo", 9096)
sizes = ising.wolff()

In [ ]:
plotting.show_history(ising.energy_history[:10]/ising.no_of_sites, "Energy")
plotting.show_history(ising.magnetization_history[:10]/ising.no_of_sites, "Magnetization")

In [ ]:
%lprun -f ising.metropolis ising.metropolis()

In [ ]:
simulation.simulation_range("ising", "wolff", [10, 20, 40], 1, "lo", 10000, 65536, 2, 3, step=0.1)

In [ ]:
ising = ising_model.IsingModel(40, 1, 3, "lo", 70536)
%timeit -n 1 -r 1 sizes = ising.wolff()

In [ ]:
np.linspace(2, 3, 11)